In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
merged_df = pd.read_csv('finaldata/merged_df.csv')
merged_df.drop(columns=merged_df[['PM2.5', 'CO', 'SO2', 'PN']], inplace=True)

In [ ]:
# Including 'Street' in the subset for plotting
merged_df['Datum'] = pd.to_datetime(merged_df['Datum'])
time_series_data_with_street = merged_df[['Datum', 'Street', 'TotalDailyTraffic', 'NO2', 'PM10']]

# Function to plot time series for each unique street
def plot_time_series_for_streets(data, column, title, color):
    streets = data['Street'].unique()
    n_streets = len(streets)

    # Creating subplots for each street
    fig, axs = plt.subplots(n_streets, 1, figsize=(15, 4 * n_streets))

    for i, street in enumerate(streets):
        street_data = data[data['Street'] == street]
        axs[i].plot(street_data['Datum'], street_data[column], label=street, color=color)
        axs[i].set_title(f'{title} for {street}')
        axs[i].set_xlabel('Date')
        axs[i].set_ylabel(column)
        axs[i].legend()
        axs[i].xaxis.set_major_locator(mdates.YearLocator())
        axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

    # Adjusting layout
    plt.tight_layout()

    # Showing the plots
    plt.show()

# Plotting time series for 
plot_time_series_for_streets(time_series_data_with_street, 'TotalDailyTraffic', 'Total Daily Traffic', 'blue')
plot_time_series_for_streets(time_series_data_with_street, 'NO2', 'NO2 Levels', 'red')
plot_time_series_for_streets(time_series_data_with_street, 'PM10', 'PM10 Levels', 'green')

In [ ]:
#Adding meteo data to merged df

def import_and_transform_meteo_data():
    streets = ['Zch_Rosengartenstrasse', 'Zch_Stampfenbachstrasse', 'Zch_Schimmelstrasse']
    street_dataframes = []

    for street in streets:
        yearly_data = []

        for year in range(2012, 2023):
            file_path = f'meteo/ugz_ogd_meteo_d1_{year}.csv'
            data = pd.read_csv(file_path)

            # Filter for a specific street
            street_data = data[data['Standort'] == street]

            # Pivot the data
            street_wide = street_data.pivot(index='Datum', columns='Parameter', values='Wert')

            # Add the year and street columns
            street_wide['Year'] = year
            street_wide['Street'] = street

            yearly_data.append(street_wide)

        # Concatenate all years' data for the street
        combined_street_data = pd.concat(yearly_data)
        street_dataframes.append(combined_street_data)

    # Concatenate all streets' data into one DataFrame
    combined_data = pd.concat(street_dataframes)

    return combined_data

meteo = import_and_transform_meteo_data()
#meteo.to_csv('meteo/meteo.csv')

meteo = pd.read_csv('meteo/meteo.csv')
meteo['Datum'] = pd.to_datetime(meteo['Datum']).dt.date
#meteo.to_csv('meteo/meteo_dt.csv', index=False)

meteo_dt = pd.read_csv('finaldata/meteo_dt.csv')

merged_meteo = pd.merge(merged_df, meteo_dt, how="left", on=['Datum', 'Street'])
#merged_meteo.to_csv('finaldata/merged_meteo.csv')

In [4]:
merged_meteo = pd.read_csv('/Users/fredericksafian/VSCODE/finaldata/merged_meteo.csv')
merged_meteo.drop(columns=merged_meteo[['Unnamed: 0', 'Year']], inplace=True)
merged_meteo['Datum'] = pd.to_datetime(merged_meteo['Datum']).dt.date

# Adding season dummies
def get_month(date):
    month_dict = {
        1: 'Jan',
        2: 'Feb',
        3: 'Mar',
        4: 'Apr',
        5: 'May',
        6: 'Jun',
        7: 'Jul',
        8: 'Aug',
        9: 'Sep',
        10: 'Oct',
        11: 'Nov',
        12: 'Dec'
    }
    return month_dict[date.month]

merged_meteo['Month'] = merged_meteo['Datum'].apply(get_month)
month_dummies = pd.get_dummies(merged_meteo['Month'])
merged_meteo = pd.concat([merged_meteo, month_dummies], axis=1).drop(columns=['Month'])

#adding weekday dummies 

merged_meteo['Weekday'] = pd.to_datetime(merged_meteo['Datum']).dt.dayofweek
weekday_dummies = pd.get_dummies(merged_meteo['Weekday'], prefix='Weekday')
merged_meteo = pd.concat([merged_meteo, weekday_dummies], axis=1)
merged_meteo.drop(['Weekday'], axis=1, inplace=True)

In [5]:
#adding tree and green spaces

#calculating trees around measurement points
import re

tree_locations_path = '/Users/fredericksafian/VSCODE/streetdata/gsz.baumkataster_baumstandorte.csv'
tree_locations_df = pd.read_csv(tree_locations_path)

def extract_coordinates(point_str):
    numbers = re.findall(r"[-+]?\d*\.\d+|\d+", point_str)
    return tuple(map(float, numbers))

tree_locations_df['coordinates'] = tree_locations_df['geometry'].apply(extract_coordinates)

def calculate_distance(coord1, coord2):
    return ((coord1[0] - coord2[0]) ** 2 + (coord1[1] - coord2[1]) ** 2) ** 0.5

target_coords = {
    "Zch_Rosengartenstrasse": (2682095, 1249940),
    "Zch_Schimmelstrasse": (2681950, 1247250),
    "Zch_Stampfenbachstrasse": (2683140, 1249040)
}

tree_counts_df = pd.DataFrame(columns=['Street', '1.5km', '1km', '500m', '200m'])

for street, coords in target_coords.items():
    tree_counts = {
        'Street': street,
        '1.5km': sum(calculate_distance(tree_coord, coords) <= 1500 for tree_coord in tree_locations_df['coordinates']),
        '1km': sum(calculate_distance(tree_coord, coords) <= 1000 for tree_coord in tree_locations_df['coordinates']),
        '500m': sum(calculate_distance(tree_coord, coords) <= 500 for tree_coord in tree_locations_df['coordinates']),
        '200m': sum(calculate_distance(tree_coord, coords) <= 200 for tree_coord in tree_locations_df['coordinates'])
    }
    tree_counts_df = pd.concat([tree_counts_df, pd.DataFrame([tree_counts])], ignore_index=True)




# calculating tree spaces around measurement points

green_spaces_path = '/Users/fredericksafian/VSCODE/streetdata/gsz.gruenflaechen.csv'
green_spaces_df = pd.read_csv(green_spaces_path)

green_spaces_df['coordinates'] = green_spaces_df['geometry'].apply(extract_coordinates)

green_space_counts_df = pd.DataFrame(columns=['Street', '1.5km', '1km', '500m', '200m'])

for street, coords in target_coords.items():
    green_space_counts = {
        'Street': street,
        '1.5km': sum(calculate_distance(green_coord, coords) <= 1500 for green_coord in green_spaces_df['coordinates']),
        '1km': sum(calculate_distance(green_coord, coords) <= 1000 for green_coord in green_spaces_df['coordinates']),
        '500m': sum(calculate_distance(green_coord, coords) <= 500 for green_coord in green_spaces_df['coordinates']),
        '200m': sum(calculate_distance(green_coord, coords) <= 200 for green_coord in green_spaces_df['coordinates'])
    }
    green_space_counts_df = pd.concat([green_space_counts_df, pd.DataFrame([green_space_counts])], ignore_index=True)



#merging it with the data set

merged_meteo = pd.merge(merged_meteo, green_space_counts_df, on='Street', how='left', suffixes=('_green', ''))
merged_meteo = pd.merge(merged_meteo, tree_counts_df, on='Street', how='left', suffixes=('','_trees'))
distance_columns_to_rename = {
    '1.5km': '1.5km_green',
    '1km': '1km_green',
    '500m': '500m_green',
    '200m': '200m_green'
}
merged_meteo = merged_meteo.rename(columns=distance_columns_to_rename)

In [6]:
#windspeed and humidity data is only in the hourly meteo set, so we will add that

def import_and_transform_meteo_hourly_data():
    streets = ['Zch_Rosengartenstrasse', 'Zch_Stampfenbachstrasse', 'Zch_Schimmelstrasse']
    street_dataframes = []
    parameters = ['WD', 'WVv', 'WVs', 'Hr']

    for year in range(2012, 2023): 
        for street in streets:
            file_path = f'/Users/fredericksafian/VSCODE/meteohourly/ugz_ogd_meteo_h1_{year}.csv'  
            data = pd.read_csv(file_path)

            # Filter for a specific street
            street_data = data[data['Standort'] == street]

            # Pivot the data to wide format
            street_wide = street_data.pivot(index='Datum', columns='Parameter', values='Wert')

            # Reset index to make 'Datum' a column again
            street_wide.reset_index(inplace=True)

            # Convert 'Datum' to datetime and keep only the date part
            street_wide['Datum'] = pd.to_datetime(street_wide['Datum']).dt.date

            # Select only the columns that exist in the dataset
            cols_to_keep = ['Datum'] + [col for col in parameters if col in street_wide.columns]
            street_wide = street_wide[cols_to_keep]

            # Group by Datum, calculate daily mean
            daily_mean = street_wide.groupby('Datum', as_index=False).mean()

            # Add the street name to the DataFrame
            daily_mean['Street'] = street

            street_dataframes.append(daily_mean)

    # Concatenate all streets' data into one DataFrame
    combined_data = pd.concat(street_dataframes, ignore_index=True)

    return combined_data

# Use this function and export the result if needed
meteo_hourly = import_and_transform_meteo_hourly_data()



#merge it

merged_meteo = pd.merge(merged_meteo, meteo_hourly, how="left", on=['Datum', 'Street'])

In [6]:
#checking the speed limits 

speed = pd.read_csv('/Users/fredericksafian/VSCODE/streetdata/taz.view_geoserver_tempo_ist_e.csv')
speed.rename(columns={'lokalisationsname': 'Street'}, inplace=True)
streets = ['Rosengartenstrasse', 'Schimmelstrasse', 'Stampfenbachstrasse']

speed = speed[speed['Street'].isin(streets)]
speed

#all the same anyway

,id,Street,messwert_von,messwert_bis,temporegime,temporegime_technical,umgesetzt_datum,rechtskraeftig_datum,publiziert_vsi_datum,ausnahmen_fahrverbot,fahrverbot_ssv,objectid
2013,619,Schimmelstrasse,0.000,261.419,T50,T50,NaN,NaN,NaN,NaN,NaN,7350
3420,1246,Rosengartenstrasse,0.000,461.979,T50,T50,NaN,NaN,NaN,NaN,NaN,8757
4052,5106,Stampfenbachstrasse,404.849,1298.793,T50,T50,NaN,NaN,NaN,NaN,NaN,9389
4509,4527,Stampfenbachstrasse,0.000,98.543,T50,T50,NaN,NaN,NaN,NaN,NaN,9846
4510,4528,Stampfenbachstrasse,98.543,404.849,T30,T30,2.018110e+13,NaN,NaN,NaN,NaN,9847
5162,5183,Rosengartenstrasse,461.979,506.905,T30,T30,NaN,NaN,NaN,NaN,NaN,10499
5163,5184,Rosengartenstrasse,623.947,858.376,T30,T30,NaN,NaN,NaN,NaN,NaN,10500
5164,5185,Rosengartenstrasse,506.905,623.947,T20,T20,2.023051e+13,2.023011e+13,2.022121e+13,NaN,NaN,10501


In [9]:
# Create the NO2_tomorrow (target variable) column by shifting the NO2 column up by one row
unique_streets = merged_meteo['Street'].unique()
data_separated = []
for street in unique_streets:
    df_street = merged_meteo[merged_meteo['Street'] == street].copy()
    df_street['NO2_tomorrow'] = df_street['NO2'].shift(-1)
    data_separated.append(df_street)

merged_meteo = pd.concat(data_separated)

merged_meteo.head()



#merged_meteo.to_csv('/Users/fredericksafian/VSCODE/finaldata/merged_with_dummies.csv')


In [13]:
merged_with_dummies = pd.read_csv('/Users/fredericksafian/VSCODE/finaldata/merged_with_dummies.csv')
merged_with_dummies.columns.unique()

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Datum', 'TotalDailyTraffic', 'Street',
       'NO', 'NO2', 'NOx', 'O3', 'O3_max_h1', 'O3_nb_h1>120', 'PM10', 'PM2.5',
       'CO', 'SO2', 'PN', 'RainDur', 'T', 'T_max_h1', 'p', 'StrGlo', 'Apr',
       'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct',
       'Sep', 'Weekday_0', 'Weekday_1', 'Weekday_2', 'Weekday_3', 'Weekday_4',
       'Weekday_5', 'Weekday_6', '1.5km_green', '1km_green', '500m_green',
       '200m_green', '1.5km_trees', '1km_trees', '500m_trees', '200m_trees',
       'WD', 'WVv', 'WVs', 'Hr', 'NO2_tomorrow'],
      dtype='object')